In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Cargar el archivo de Excel
excel_path = 'C:/Users/Acer/Desktop/PythonNotes/Python/DataSetsPython/061524 ESTADOS RESTANTE.xlsx'
df = pd.read_excel(excel_path, sheet_name='ZACATECAS 2023')

# Configuración de Selenium
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

In [ ]:
# Lista para almacenar los datos extraídos
data = []

# Iterar sobre cada URL en la columna 'Dirección del anuncio'
for index, url in enumerate(df['Dirección del anuncio'], start=1):
    driver = setup_driver()
    driver.get(url)
    driver.implicitly_wait(30)  # Esperar a que la página cargue completamente
    print(f"Procesando URL {index} de {len(df)}: {url}")
    # Selector base para la tabla

    base_selector = "#p-tabpanel-5 > div > app-sitiopublico-detalle-economicos-pc > div > div > app-sitiopublico-detalle-economicos-list > div.p-col-12.p-sm-12.p-md-12.p-lg-12.ng-star-inserted > p-table > div > div > div > div.p-datatable-scrollable-body.ng-star-inserted > table > tbody > tr"
    rows = driver.find_elements(By.CSS_SELECTOR, base_selector)
    if not rows:
        base_selector = "#p-tabpanel-5 > div > app-sitiopublico-detalle-economicos-pc > div > div > app-sitiopublico-detalle-economicos-list > div.p-col-12.p-sm-12.p-md-12.p-lg-12.ng-star-inserted > p-table > div > div > table > tbody > tr"
        rows = driver.find_elements(By.CSS_SELECTOR, base_selector)

    for row in rows:
        description = row.find_element(By.CSS_SELECTOR, 'td:nth-child(4)').text
        unit = row.find_element(By.CSS_SELECTOR, 'td:nth-child(6)').text
        try:
            quantity = row.find_element(By.CSS_SELECTOR, 'td:nth-child(7)').text
        except Exception:
            quantity = "N/A"  # Usa "N/A" o cualquier otro valor que represente un dato no disponible

        data.append({'Dirección del anuncio': url, 'Descripción CUCoP+': description, 'Cantidad': quantity, 'Unidad de medida': unit})
        print(f"Extraído: Descripción - {description}, Unidad - {unit}, Cantidad - {quantity}")

    driver.quit()

    if index % 90 == 0:
        temp_df = pd.DataFrame(data)
        temp_df.to_excel(f'C:/Users/Acer/Desktop/PythonNotes/Python/DataSetsPython/ZACATECAS_temp_{index}.xlsx', index=False)
        print(f"Datos guardados temporalmente en ZACATECAS_temp_{index}.xlsx")

# Convertir los datos a DataFrame
new_df = pd.DataFrame(data)

# Guardar en nuevo archivo Excel
new_df.to_excel('C:/Users/Acer/Desktop/PythonNotes/Python/DataSetsPython/ZACATECAS_actualizado.xlsx', index=False)

print("Proceso completado. El archivo Excel ha sido actualizado.")